# 이미지에 대한 벡터 표현 생성

## 필수 라이브러리 설치

In [ ]:
!pip install sentence_transformers elasticsearch

In [ ]:
import getpass
import torch
import os
import torchvision.transforms as transforms
import json
from PIL import Image
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch, helpers

## 샘플 이미지 다운로드

In [ ]:
!curl -LJO https://raw.githubusercontent.com/PacktPublishing/Vector-Search-with-Elastic/main/chapter5/images/images.tar
!tar xvf ./images.tar

In [ ]:
# 이미지를 포함하는 폴더 경로 지정 
image_dir = './images/index'

# 인덱스 명 정의
index_name = 'images_book_demo'

# 일래스틱 클라우드에 접속 
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID: ') 
es_api_id = getpass.getpass('Enter cluster API key ID: ') 
es_api_key = getpass.getpass('Enter cluster API key: ')

es = Elasticsearch(cloud_id=es_cloud_id, api_key=(es_api_id, es_api_key))
es.info() # 클러스터 정보 확인

In [ ]:
# 이미지를 임베딩 할 수 있는 모델 다운로드 및 로드 
model = SentenceTransformer('clip-ViT-B-32-multilingual-v1')

# 이미지 전처리 함수 구성
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    lambda image: image.convert("RGB"),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [ ]:
def create_mapping_if_new(index_name, es):

    # 맵핑 정의
    mapping = {
      "mappings": {
        "properties": {
          "image_vector": {
            "type": "dense_vector",
            "dims": 512,
            "index": True,
            "similarity": "cosine"
          } ,
          "filename": {
            "type": "keyword"
          }
      }
    }
  }

    # 동일한 이름의 인덱스가 존재하는지 체크
    if not es.indices.exists(index=index_name):
        # 정의된 맵핑으로 인덱스 생성 
        es.indices.create(index=index_name, body=mapping)

def embed_image(image_path):
    # 이미지 파일 오픈 
    with Image.open(image_path) as img:
        # 앞서 정의한 방식으로 이미지 전처리 
        image = transform(img).unsqueeze(0)

        # 컴퓨터 환경을 체크하여 가능한 경우 GPU 활용 
        if torch.cuda.is_available():
            image = image.to('cuda')
            model.to('cuda')

        # 모델을 활용하여 이미지 벡터 생성
        image_vector = model.encode(image)

        # 이미지 벡터가 torch tensor 형태로 생성된 경우, CPU에서 처리 가능하도록 데이터를 변환
        if isinstance(image_vector, torch.Tensor):
            image_vector = image_vector.cpu().numpy()

        # 리스트 형태로 변환 후,
        image_vector = image_vector.tolist()

        # 이미지 벡터 반환
        return image_vector

In [ ]:
# 동일한 인덱스가 존재하지 않는 경우, 정의된 맵핑으로 인덱스 생성 
create_mapping_if_new(index_name, es)

# 이미지 파일과 벡터를 저장할 딕셔너리 형태의 변수 선언
data = {}

# 폴더에 있는 이미지 순환 
for image_file in os.listdir(image_dir):
    # 이미지 벡터 생성 
    image_vector = embed_image(os.path.join(image_dir, image_file))

    # 이미지 벡터를 data 딕셔너리에 저장 
    data[image_file] = image_vector[0]

# 일래스틱서치에 이미지 벡터를 색인
documents = []
for filename, vector in data.items():

    # 색인 대상 문서 구성
    document = {'_index': index_name,
                '_source': {"filename": filename,
                            "image_vector": vector
                    }
          }


    documents.append(document)

#documents

In [ ]:
from elasticsearch.helpers import BulkIndexError

# bulk API를 통해 앞서 구성한 doc를 대량 색인
try:
  helpers.bulk(es, documents)
except BulkIndexError as e:
  for x in e.errors:
    print(x)

# kNN 검색

검색 쿼리용 이미지 벡터화

In [ ]:
search_image = './images/search/patrice-bouchard-Yu_ejF2s_dI-unsplash.jpg'
search_image_vector = embed_image(search_image)

kNN 벡터 검색 수행

In [ ]:
knn = {
    "field": "image_vector",
    "query_vector": search_image_vector[0],
    "k": 1,
    "num_candidates": 10
  }
fields = ["filename"]
size = 1
source = False

In [ ]:
results = es.search(index=index_name,
                    knn=knn,
                    source=source,
                    fields=fields,
                    size=size
                  )

In [ ]:
result_filename = results['hits']['hits'][0]['fields']['filename'][0]

## 검색 결과 이미지 표시

In [ ]:
from IPython.display import Image
Image('./images/index/'+result_filename, width=400)